In [1]:
import time
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
topk = 10
rank_list = [i+1 for i in range(topk)]

In [3]:
%%time
ML100K_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
dataset = pd.read_csv(
  ML100K_URL, 
  names=["user_id", "item_id", "rating", "timestamp"], 
  sep="\t"
)

CPU times: user 44.7 ms, sys: 14 ms, total: 58.6 ms
Wall time: 440 ms


# User2User

In [4]:
%%time
user_data = csr_matrix(
  (dataset.rating, 
  (dataset.user_id, dataset.item_id))
)
user_similarity = cosine_similarity(user_data)

CPU times: user 129 ms, sys: 7.85 ms, total: 137 ms
Wall time: 161 ms


In [5]:
user_similarity[1:6, 1:6]

array([[1.        , 0.16693098, 0.04745954, 0.06435782, 0.37847518],
       [0.16693098, 1.        , 0.11059132, 0.17812119, 0.07297896],
       [0.04745954, 0.11059132, 1.        , 0.34415072, 0.02124453],
       [0.06435782, 0.17812119, 0.34415072, 1.        , 0.03180425],
       [0.37847518, 0.07297896, 0.02124453, 0.03180425, 1.        ]])

In [6]:
%%time
# ユニークなユーザーIDのリストを作成する
uq_users = np.sort(dataset.user_id.unique().tolist())

#類似ユーザー情報を格納するテーブルを作成する
df_similar_user_list = pd.DataFrame(
  columns=[
    'base_user_id',    #対象ユーザー
    'similar_user_id', #類似ユーザー
    'cosine_score',    #コサイン類似度
    'rank'             #順位
  ]
)

# 各ユーザー(対象ユーザー)に対して類似ユーザーを絞り込む
for user_id in uq_users:

  # 対象ユーザーから見た類似度スコア情報を取得
  similar_score = user_similarity[user_id]

  df_similar_user = pd.DataFrame()
  df_similar_user['base_user_id'] = [user_id] * topk
  # コサイン類似度からトップ10類似ユーザーを取得
  df_similar_user['similar_user_id'] = \
    np.argsort(similar_score)[::-1][1:topk+1]
  # トップ10の類似ユーザーのコサイン類似度を取得
  df_similar_user['cosine_score'] = \
    np.sort(similar_score)[::-1][1:topk+1]

  # 全体テーブルに格納する
  df_similar_user['rank'] = rank_list
  df_similar_user_list = \
    df_similar_user_list.append(
      df_similar_user, 
      ignore_index=True
    )

CPU times: user 3.79 s, sys: 34.5 ms, total: 3.82 s
Wall time: 4.59 s


In [7]:
df_similar_user_list

,base_user_id,similar_user_id,cosine_score,rank
0,1,916,0.569066,1
1,1,864,0.547548,2
2,1,268,0.542077,3
3,1,92,0.540534,4
4,1,435,0.538665,5
...,...,...,...,...
9425,943,276,0.498243,6
9426,943,709,0.493219,7
9427,943,586,0.491972,8
9428,943,472,0.488882,9


# Item2Item

In [8]:
%%time
item_data = csr_matrix(
  (dataset.rating, 
  (dataset.item_id, dataset.user_id))
)
item_similarity = cosine_similarity(item_data)

CPU times: user 176 ms, sys: 12.6 ms, total: 188 ms
Wall time: 452 ms


In [9]:
item_similarity[1:6, 1:6]

array([[1.        , 0.40238218, 0.33024479, 0.45493792, 0.28671351],
       [0.40238218, 1.        , 0.27306918, 0.50257077, 0.31883618],
       [0.33024479, 0.27306918, 1.        , 0.32486639, 0.21295656],
       [0.45493792, 0.50257077, 0.32486639, 1.        , 0.33423948],
       [0.28671351, 0.31883618, 0.21295656, 0.33423948, 1.        ]])

In [10]:
%%time
# ユニークなアイテムIDのリストを作成する
uq_items = np.sort(dataset.item_id.unique().tolist())

#類似アイテム情報を格納するテーブルを作成する
df_similar_item_list = pd.DataFrame(
  columns=[
    'base_item_id',    #対象アイテム
    'similar_item_id', #類似アイテム
    'cosine_score',    #コサイン類似度
    'rank'             #順位
  ]
)

# 各アイテム(対象アイテム)に対して類似アイテムを絞り込む
for item_id in uq_items:

  # 対象アイテムから見た類似度スコア情報を取得
  similar_score = item_similarity[item_id]

  df_similar_item = pd.DataFrame()
  df_similar_item['base_item_id'] = [item_id] * topk
  # コサイン類似度からトップ10類似アイテムを取得
  df_similar_item['similar_item_id'] = \
    np.argsort(similar_score)[::-1][1:topk+1]
  # トップ10の類似アイテムのコサイン類似度を取得
  df_similar_item['cosine_score'] = \
    np.sort(similar_score)[::-1][1:topk+1]
  df_similar_item['rank'] = rank_list

  # 全体テーブルに格納する
  df_similar_item_list = \
    df_similar_item_list.append(
      df_similar_item, 
      ignore_index=True
    )

CPU times: user 10.2 s, sys: 40.1 ms, total: 10.2 s
Wall time: 24.1 s


In [11]:
df_similar_item_list

,base_item_id,similar_item_id,cosine_score,rank
0,1,50,0.734572,1
1,1,181,0.699925,2
2,1,121,0.689786,3
3,1,117,0.664555,4
4,1,405,0.641322,5
...,...,...,...,...
16815,1682,1268,0.266207,6
16816,1682,1113,0.248069,7
16817,1682,1428,0.231455,8
16818,1682,960,0.229794,9
